In [2]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

# города, которые надо посетить
places = ['Norilsk', 'Novosibirsk', 'Samara', 'Peterhof', 'Moscow',
          'Krasnodar', 'Belgorod', 'Murmansk',
          'Vladivostok', 'Tuyumen', 'Novokuznetsk', 'Ufa',
          'Shadrinsk', 'St Petersburg']

# расстояния между городами в верхней треугольной матрице
# тут они могут быть не целыми (для милп), но как подсказал гугл по расстояниям (у нас тут пока только целые):
dists = [[1619, 2633, 2836, 2878, 3826, 3383, 2105, 3816, 1745, 1734, 2286, 1906, 2814],
         [2129, 3129, 2811, 3267, 3094, 2910, 3716, 1101, 307, 1714, 1215, 3106],
         [1436, 853, 1213, 969, 1963, 5831, 1071, 2424, 419, 926, 1417],
         [647, 1758, 1115, 1021, 6561, 2065, 3434, 1653, 2009, 40],
         [1195, 577, 1487, 6417, 1710, 3117, 1164, 1611, 634],
         [643, 2683, 6982, 2269, 3543, 1618, 2113, 1754],
         [2052, 6781, 2014, 3392, 1380, 1882, 1111],
         [5926, 2056, 3189, 1961, 2083, 1012],
         [7676, 1407, 2012, 506, 1986],
         [1407, 653, 166, 2042],
         [2012, 1517, 3411],
         [506, 1631],
         [1986],
         []]

# задаем количество узлов n и список вершин V
n, V = len(dists), set(range(len(dists)))

# матрица расстояний
c = [[0 if i == j
      else dists[i][j-i-1] if j > i  # потому что треугольную задаем
      else dists[j][i-j-1]
      for j in V] for i in V]

model = Model() # задаем модель смешанного целочисленного линейного программирования

# задаем бинарные переменные, показывающие, будет ли дуга (i,j) использована в общем пути или нет
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# чтобы избежать повторений в пути: каждый город должен иметь различный id
# в построенном маршруте кроме первого города
y = [model.add_var() for i in V]

# целевая функция: минимизируем расстояния
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

# вводим ограничение: покидаем один город только 1 раз
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# вводим ограничение: входим в один город только 1 раз
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# исключаем повторения
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# оптимизация
model.optimize()

# проверяем, найдено ли решение, выводим
if model.num_solutions:
    out.write('Route with total distance %g found: %s'
              % (model.objective_value, places[0]))
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % places[nc])
        if nc == 0:
            break
    out.write('\n')

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 184 (0) rows, 195 (-15) columns and 832 (0) elements
Clp1000I sum of infeasibilities 1.55948e-05 - average 8.47546e-08, 148 fixed columns
Coin0506I Presolve 166 (-18) rows, 29 (-166) columns and 372 (-460) elements
Clp0006I 0  Obj 10328.933 Primal inf 2.1756952e-06 (2) Dual inf 2.0000008e+08 (16)
Clp0029I End of values pass after 29 iterations
Clp0014I Perturbing problem by 0.001% of 5.9901703 - largest nonzero change 3.0227567e-05 ( 0.00065501579%) - largest zero change 2.623149e-05
Clp0000I Optimal - objective value 10328.933
Clp0000I Optimal - objective value 10328.933
Coin0511I After Postsolve, objective 10328.933, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 10328.933 Dual inf 91.999998 (2)
Clp0014I Perturbing problem by 0.001% of 0.96923107 - largest nonzero change 1.932281e-05 ( 0.0009994557%) - largest zero change 1.5272747e-05
Clp0000I Optimal - objective value